# Курс "Программирование на языке Python. Уровень 4. Анализ и визуализация данных на языке Python. Библиотеки numpy, pandas, matplotlib"

## Модуль 5. Библиотека pandas. Работа с датасетами.

- Загрузка датасетов
- Обработка отсутствующих данных
- Поиск и удаление дублей
- Создание новых признаков, функции ```apply()``` и ```applymap()```
- Категориальные признаки, функция ```cut()```, dummy-признаки
- Горизонтальные и вертикальные объединения, функции ```merge()``` и ```concat()```
- "Широкий" и "Длинный" форматы таблиц (stack/unstack)
- Сохранение датасетов



In [1]:
# загрузите необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Загрузка датасетов

Pandas поддерживает загрузку данных из множества источников. Чаще всего придется работать с данными в форматах CSV, XLSX и JSON, а также загружать их из базы данных.

Рассмотрим загрузку данных из файла формата csv - данных, разделенных запятыми. Посмотрим содержимое файла, который мы будем загружать:

In [ ]:
with open('data/load_example1.csv') as f:
    print(f.read())

Для загрузки будем использовать функцию ```pd.read_csv()```. Укажите в качестве параметра имя файла.

In [ ]:
df = pd.read_csv('data/load_example1.csv')
df

Обратите внимание, как ведет себя функция по умолчанию:
 - названия колонок соответствуют содержимому первой строки файла
 - индекс по умолчанию - последовательность чисел.
 
Чтобы ```read_csv()``` включила первую строку в наш DataFrame, передайте ей параметр ```header=None```:

In [ ]:
df = pd.read_csv('data/load_example1.csv', header=None)
df

Также можно задать названия столбцов самостоятельно:

In [ ]:
df = pd.read_csv('data/load_example1.csv', names=['aa', 'bb', 'cc', 'dd', 'mmessage'])
df

Чтобы указать, что один из столбцов - индекс, используйте параметр index_col, там можно указать либо название поля, либо его порядковый номер:

In [ ]:
df = pd.read_csv('data/load_example1.csv', index_col='message')
df

In [ ]:
# или 
df = pd.read_csv('data/load_example1.csv', index_col=4)
df

Чтобы пропустить те или иные строки, используйте параметр ```skiprows```, ему можно передать список строк, которые надо пропустить:

In [ ]:
df = pd.read_csv('data/load_example1.csv', skiprows=[0,1])
df

Обратите внимание: указанные строки вообще не участвуют в разборе файла!

При разборе CSV-файлов также могут встретиться следующие трудности:
 - вместо отсутствующих данных могут быть строки типа "NULL", "n/a" и т.п.
 - разделителями могут быть символы ";" (особенно при выгрузке данных из русской версии Microsoft Excel), или же символ табуляции.
 
Со всем этим может справиться функция ```read_csv()```. Загрузим файл ```data/load_example2.csv```

In [ ]:
with open('data/load_example2.csv') as f:
    print(f.read())

 Для указания символа ";" в качестве разделителя, передайте фукнции параметр ```sep=';'```

In [ ]:
df = pd.read_csv('data/load_example2.csv', sep=';')
df

Чтобы обработать строки "данные отсутствуют" в данном примере, функции ```read_csv()``` нужно передать параметр ```na_values='данные отсутствуют'```

In [ ]:
df = pd.read_csv('data/load_example2.csv', sep=';', na_values='данные отсутствуют')
df

### Обработка отсутствующих данных

С отсутствующими данными в объекте Series можно сдедать следующее:
 - удалить функцией ```.dropna()```
 - заполнить подходящим значением, используя функцию ```.fillna()```.
 
Для поиска пустых значений используем функцию ```.isnull()```.
 
Посмотрим, как это работает на примере первого сета. Снова загрузим его.

In [ ]:
df = pd.read_csv('data/load_example1.csv', index_col='message')
df

Получить series из позиций в 'b', содержащих NaN, можно используя булеву маску по колонке "b":

In [ ]:
df['b'][df['b'].isnull()]

Посмотрим, как работает ```.dropna()``` в Series, получим колонку 'b' в виде этого объекта:

In [ ]:
b = df['b'].copy()
b

Вызовем ```dropna()```:

In [ ]:
bbd = b.dropna()
bbd

Заполним отсутствующие значения

In [ ]:
# можно заполнить конкретным значением
bbf = b.fillna(0)
bbf

In [ ]:
# а можно средним по всей Series
bbf = b.fillna(b.mean())
bbf

В случае с DataFrame это работает похожим образом, только функция удаляет строки, в которых встречается хотя бы одно незаполненное значение:

In [ ]:
df.dropna()

Этой функции можно задать порог срабатывания, в зависимости от количества __заполненных подряд значений__ в строке. Например, нам нужно удалить только те строки, в которых заполнены как минимум первые три значения подряд:

In [ ]:
df.dropna(thresh=3)

Обратите внимание на строку "bar" - несмотря на незаполненную ячейку, она не попала под удаление!

Также можно заполнять отсутствующие данные числами:

In [ ]:
df.fillna(100500)

Эта функция работает и для заполнения "пробелов" горизонтальными/вертикальными агрегатными вычислениями.

In [ ]:
df.fillna(df.mean())

Чтобы эти функции отработали внутри самого объекта и не возвращали его копию, используйте параметр ```inplace=True```.

__ЗАДАНИЕ__ Замените отсутствующие значения в колонке b на среднее по ней, c - на 0, d - на среднее по всей матрице.

In [ ]:
# ваш код здесь

### Поиск и удаление дублей

Проверить, является ли уникальным индекс, можно, опросив свойство индекса ```is_unique```:

In [ ]:
df.index.is_unique

Получить булеву маску для дубликатов по индексу можно, вызвав метод ```.duplicated()```. Применение отрицания этой маски вернет DataFrame без строки с дублированным индексом.

In [ ]:
df[~df.index.duplicated()]

Тем же методом объекта DataFrame или Series можно получить булеву маску для дубликатов записей в датасете:

In [ ]:
df.duplicated()

Методу можно передать параметр ```keep=```, который не будет отмечать признаком True либо первый дубликат (значение first), либо последний (значение last).

In [ ]:
df.duplicated(keep='last')

Метод можно вызвать, передав ему список признаков, в которм нужно ограничиться поиском дубликатов:

In [ ]:
df.duplicated(['b'])

Удалить дубликаты можно функцией ```drop_duplicates()```. Она работает так же, как и ```duplicated()```, но она возвращает новый DataFrame без дубликатов. Ее можно вызвать с параметром inplace().

In [ ]:
df.drop_duplicates()

In [ ]:
df[5, 'b'] = 10
df.duplicated()

### Создание новых признаков, функции apply() и applymap()

С созданием новых признаков на базе существующих данных мы уже знакомы, но часто бывает так, что для вычисления новых признаков нужно применить более сложные процедуры, чем стандартные. Для этого существуют функции ```apply()``` и ```applymap()```.



In [ ]:
df = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
df

Посмотрим, как работает метод ```apply()```. Функция, которая указана в качестве параметра этого метода принимает на вход объект Series - столбец и возвращает значение, которое объединяется в объект Series, структцрно соответствующий строке текущего DataFrame. Для вычисления по строкам и формирования столбцов функции ```apply()``` нужно передать параметр ```axis=1```

In [ ]:
# Функции f и ff эквивалентны:
def f(x):
    print(x)
    return x.max() - x.min()

ff = lambda x: x.max() - x.min()

df.apply(f, axis=1)

Добавление нового вычисленного признака теперь будет выглядеть так:

In [ ]:
df['diff'] = df.apply(f, axis=1)
df

В отличие от ```apply()```, ```applymap()``` вычисляется для каждого элемента и возвращает значение, которое должно быть установлено на его место.

In [ ]:
format_ = lambda x: '%.2f' % x
df.applymap(format_)

Для того, чтобы проделать такую операцию над Series, воспользуйтесь функцией map():

In [ ]:
# df['e'] = df['e'].map(format_)
df
df['e'] = df['e'].map(lambda x: float(x))

__ЗАДАНИЕ__ В текущий DataFrame ```df``` добавьте строку с суммами значений 1000, если значение больше нуля, и 0 в противном случае.

In [ ]:
# ваш код здесь

__ЗАДАНИЕ__  В датасете Titanic проверьте признак "Возраст"("Age") на выбросы (отрицательный возраст, посмотрите максимальный возраст - он правдоподобен?).
Если там есть отсутствующие значения - на их место поставьте медианный возраст пассажиров.

In [ ]:
# загрузите Titanic
df_titanic = pd.read_csv('data/titanic.csv',
                  index_col='PassengerId')

# ваш код здесь


### Категориальные признаки, функция cut(), dummy-признаки

Часто возникает задача сделать более точным один из признаков, сократив по нему количество возможных вариантов, а то и вообще сведя к одному или нескольким булевам признакам (dummy-признакам).

Это может быть применено к различным количественным характеристикам (например, возраст, вес - "несовершеннолетний"/"толстый"), к географическим признакам ("Москва"/"не Москва"), к временным признакам ("До Революции/После Революции") и т.д.

Рассмотрим создание категориальных признаков на примере работы с датасетом "Титаник".

**Создадим признак "Возрастная категория"**

Создавать будем двумя способами: 
1. с помощью функции, которая возвращает 1, если до 30-ти, 2, если от 30-ти до 55-ти и 3, если старше 55.
2. с помощью функции ```pd.cut()```

In [ ]:
def age_category(age):
    '''
    < 30 -> 1
    >= 30, <55 -> 2
    >= 55 -> 3
    '''
    if age < 30:
        return 1
    elif age < 55:
        return 2
    else:
        return 3
    
df_titanic['Age_category'] = df_titanic['Age'].apply(age_category)

Теперь функцией ```cut()```:

In [ ]:
# создадим "козрзинки", в которые будем раскладывать наши категории
bins = [0,30,55,100]
age_categories = pd.cut(df_titanic['Age'], bins, right=False) # right=False - означает, что правая граница включена
age_categories

Чтобы добавить требуемые метки, передадим их в виде списка:

In [ ]:
labels = [1,2,3]
age_categories = pd.cut(df_titanic['Age'], bins, labels=labels, right=False) # right=False - означает, что правая граница включена
age_categories

Теперь добавим их в наш датасет и сравним с тем, что мы сделали с помощью функции ```apply()```:

In [ ]:
df_titanic['Age_category_1'] = pd.cut(df_titanic['Age'], bins, labels=labels, right=False)

In [ ]:
df_titanic.T.duplicated()

Еще раз посмотрим на добавленные признаки:

In [ ]:
df_titanic[ ['Age_category', 'Age_category_1']]

In [ ]:
df_titanic['Age_category_1'][0:10]

Тот признак, который мы создали из функции ```cut()``` стал категориальным - его значения могут принимать три величины: 1, 2 или 3.

#### Добавление dummy-признаков

В данном случае мы вместо одного признака "возрастная категория" с тремя возможными значениями сделаем три булевых признака. В задачах машинного обучения бывает необходимость оценить степень влияния принадлежности к той или иной группе на решение задачи, и если влияние незначительное - избавиться от такого признака. Потом, ряд алгоритмов принимает на вход только цифровые значения, и такое действие позволяет избавиться от one-hot encoding для таких признаков.

Добавить их можно очень просто: функцией ```pd.get_dummies()```. При этом признак, из которого мы получаем эти dummy-признаки, не обязательно должен быть категориальным.

In [ ]:
age_dummies = pd.get_dummies(df_titanic['Age_category'])
age_dummies

Как видно, названия колонок для этих признаков взяты из их значений. Чтобы придать им осмысленное название, пользуйтесь параметром ```prefix=```.

In [ ]:
age_dummies = pd.get_dummies(df_titanic['Age_category_1'], prefix="age_cat_")
age_dummies

Присоединить наши новые признаки к датасету можно методом ```.join()```.

In [ ]:
df_titanic = df_titanic.join(pd.get_dummies(df_titanic['Age_category_1'], prefix="age_cat_"))

In [ ]:
df_titanic

### Горизонтальные и вертикальные объединения, функции merge() и concat()

"Горизонтальные" объединения (аналог JOIN в SQL) в pandas выполняются функцией или методом ```merge()```. По умолчанию оъединение производится по колонкам с совпадающими именами и только по ключам, которые включаются в оба DataFrame'а.

Создадим DataFrame с номерами грузовиков и некоторой абстрактной статистикой по ним.

In [ ]:
trucks = ['X101AP', 'T123TM', 'X098AP', 'T123TM',  'X098AP', 'X101AP']
df_trucklog = pd.DataFrame({'truck':trucks, 'week':[12,10,5,6,7,9], 'month':[212,310,85,186,217,299]}, columns=['truck', 'week', 'month'])
df_trucklog

И создадим DataFrame со справочником по этим грузовикам, которые включают, например, марку. 

In [ ]:
df_trucks = pd.DataFrame({'plate_number': df_trucklog['truck'].unique(),
'brand': ['VOLVO', 'RENAULT', 'MAN']}, columns=['plate_number', 'brand'])
df_trucks

Предположим, заказчику захотелось увидеть в отчете по этим грузовикам не только номер, но и марку, а в изначальном датасете она отсутствует. Мы можем "вытащить" марку из справочника, выполнив функцию ```merge()```.

Укажем в параметрах названия полей, по которым надо выполнить объединение. В результате будет возвращен новый DataFrame.

In [ ]:
df_trucklog.merge(df_trucks, left_on='truck', right_on='plate_number')

Добавим в журнал по грузовикам машину, которой нет в справочнике.

In [ ]:
df_trucklog = df_trucklog.append({'week': 5, 'month': 20, 'truck':'X055XT'}, ignore_index=True)
df_trucklog

Если мы хотим, чтобы данные по этой машине также присутствовали в отчете, мы можем включить все ключи слева параметром ```how='left'```.

In [ ]:
df_trucklog.merge(df_trucks, left_on='truck', right_on='plate_number', how='left')

"Вертикальное" объединение таблиц возможно с помощью функции ```concat()```. На вход она получает список датафреймов, которые надо объединить. Если вы используете сгенерированные ключи, не забудьте указать параметр ```ignore_keys=True```.

In [ ]:
df_trucklog1 = pd.DataFrame({'truck':trucks, 'week':[2,7,6,6,2,1], 'month':[50,25,110,162,272,292]}, columns=['truck', 'week', 'month'])
df_trucklog1

In [ ]:
df_trucklog_new = pd.concat([df_trucklog, df_trucklog1], ignore_index=True)
df_trucklog_new

__???__ А как разбить DataFrame? 

__ЗАДАНИЕ__

Есть таблица студентов и номеров их зачетных книжек. Есть несколько объектов Series с оценками по различным предметам, где индексы - номера зачетных книжек. Нужно получить следующие данные:
1. Получить объединенный табель по всем предметам и студентам.
2. Получить список студентов, сдавших сессию на "хорошо" и "отлично"
3. Получить список студентов, которые сдали не все экзамены

In [ ]:
df_students = pd.DataFrame({'surname': ['Ivanov', 'Petrov', 'Sidorov', 'Kuznetsov', 'Kotova', 'Ivanov'],\
                           'logbook': ['X01', 'X02', 'X04', 'X03', 'X05', 'X05', ]})
s_physics = pd.Series([5,5,2,3,4], index=['X05', 'X02', 'X03', 'X06', 'X01', ])
s_calculus = pd.Series([4,3,5,5,4,5], index=['X02', 'X01', 'X04', 'X05', 'X06', 'X03'])
s_linalg = pd.Series([5,2,3,4], index=['X01', 'X03', 'X05', 'X06'])

# ваш код здесь


### "Раскатка" и "Штабелирование" данных

Работа со сложным индексм: функции ```stack()``` и ```unstack()```.

In [ ]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))
data


 Функция ```stack()``` "сложит" данные вертикально, построив "сложный индекс":

In [ ]:
stacked = data.stack()
stacked

Функция ```unstack()``` выполнит противоположную задачу - данные из "сложного индекса" вынесет в колонки:

In [ ]:
stacked.unstack()

#### Функции melt() и pivot()

```melt()``` - преобразует столбцы в строки, добавляя соотвествующие столбцы variable и value. \
```pivot()```- наоборот, собирает данные по строкам в столбцы

In [2]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [4]:
melted = pd.melt(df, ['key'], var_name='letters', value_name='numbers')
melted

,key,letters,numbers
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [6]:
reshaped = pd.pivot(melted, index=['key'], columns=['letters'], values='numbers')
reshaped

letters,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


#### Сохранение данных

Для сохранения датасетов в требуемом виде можно использовать следующие функции:
 - ```to_csv()``` - для сохранения данных в виде CSV
 - ```to_excel()``` - для сохранения данных в виде Excel Workbook \
...а также во можестве других форматов (см. документацию).
 
Также данные можно экспортировать в структуры Python и numpy:
 - ```to_dict()``` - этот метод вернет словарь с содержимым DataFrame, ровно в том же виде, чтобы из него можно было бы создать новый DataFrame
 - ```to_dict('records')``` - в данном случае этот метод вернет список словарей
 - ```to_numpy()``` - а этот метод можно использовать, если вам нужна матрица numpy

__ЗАДАНИЕ__: Загрузите датасет "toy_budget.csv". Он содержит информацию по доходам и расходам подразделений компании по месяцам. Преобразуйте его в датасет, в котором месяцы отложены как столбцы и упорядочены так, как это задано в списке ```months```. Подсчитайте суммы по каждому месяцу.

Указание: используйте функцию ```reindex()``` для упорядочивания колонок.

In [14]:
months = ['apr',
 'may',
 'jun',
 'jul',
 'aug',
 'sep',
 'oct',
 'nov',
 'dec',
 'jan',
 'feb',
 'mar',]

df_toy_budget = pd.read_csv('data/toy_budget.csv', sep=';', index_col=0)
df_toy_budget

,Div,Account,Type,Month,Amount
0,IT,System Costs,Cost,apr,364.68
1,IT,IT Support,Cost,apr,62.27
2,Finance,Accounting,Cost,apr,78.89
3,Finance,Billing,Cost,apr,55.81
4,Adm,Office Adm,Cost,apr,88.94
...,...,...,...,...,...
103,Adm,Office Adm,Cost,mar,88.94
104,Air,Gross Margin,Income,mar,1055.67
105,Truck,Gross Margin,Income,mar,931.48
106,Rail,Gross Margin,Income,mar,919.21


In [ ]:
# ваш код здесь